In [9]:
"""
NBA Offseason Grading System: Metric Standardization

Goal: Standardize NBA player and team metrics from an Excel workbook by converting raw statistics into z-scores.

What the script does:
    1. Loads my Excel workbook
    2. Converts text 'NaN' entries to actual missing values
    3. Converts letter grades to numeric scales
    4. Standardizes continuous metrics using z-scores (mean = 0, std = 1)
    6. Saves everything to a new Excel file 
    
Input file is an Excel workbook with 4 sheets:
    - Player Role Fit: Role-based metrics for players
    - Player Skill Fit: Skill-based metrics for players
    - Team Role Fit: Role-based metrics for teams
    - Team Skill Fit: Skill-based metrics for teams

Output file will be a new Excel file with all original columns + new standardized columns
"""

# External libraries needed for this script
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings

# Ignore warning messages
warnings.filterwarnings('ignore')

# File paths
input_file = 'data/raw/Offseason_Grades.xlsx'
output_file = 'data/processed/Offseason_Grades_Standardized.xlsx'

# Script begins
print("="*70)
print("NBA Metrics Standardization")
print("="*70)
print("\nLoading data...")

# Load data from Excel
    # Read all four sheets in the workbook
    # Each sheet becomes a separate pandas DataFrame
player_role_fit = pd.read_excel(input_file, sheet_name='Player Role Fit')
player_skill_fit = pd.read_excel(input_file, sheet_name='Player Skill Fit')
team_role_fit = pd.read_excel(input_file, sheet_name='Team Role Fit')
team_skill_fit = pd.read_excel(input_file, sheet_name='Team Skill Fit')

# Confirm data loaded successfully
# .shape returns (number_of_rows, number_of_columns) as a tuple
print("✓ Data loaded successfully!")
print(f"  Player Role Fit: {player_role_fit.shape}")
print(f"  Player Skill Fit: {player_skill_fit.shape}")
print(f"  Team Role Fit: {team_role_fit.shape}")
print(f"  Team Skill Fit: {team_skill_fit.shape}")

# Handle manual 'NaN' entries
    # My 'NaN' values in the workbook are text strings instead of actual blank cells
    # Python treats 'NaN' as text, not a missing value.
    # To avoid problems I convert 'NaN' to actual missing values
print("\n" + "="*70)
print("STEP 1: Converting manual 'NaN' text to actual NaN values")
print("="*70)

# NaN loop
    # This loops through each column in the DataFrame
    # Replaces text 'NaN' with actual NaN
    # Converts columns to numeric where possible
def convert_nan_strings(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].replace(['NaN'], np.nan)
        try:
            df[col] = pd.to_numeric(df[col], errors='ignore')
        except:
            pass
    return df

# Apply the NaN conversion function to all four DataFrames
player_role_fit = convert_nan_strings(player_role_fit)
player_skill_fit = convert_nan_strings(player_skill_fit)
team_role_fit = convert_nan_strings(team_role_fit)
team_skill_fit = convert_nan_strings(team_skill_fit)

print("✓ Converted text 'NaN' entries to actual NaN values")

# Convert letter grades to Numeric
    # Letter grades (like A+, B-, C) cannot be used directly in mathematical calculations
    # I convert them to numeric values so we can use them in the models
print("\n" + "="*70)
print("STEP 2: Converting ordinal grade columns to numeric")
print("="*70)

# Position size (PosSize) metric
possize_mapping = {
    'D': 1, 'D+': 2,
    'C-': 3, 'C': 4, 'C+': 5,
    'B-': 6, 'B': 7, 'B+': 8,
    'A-': 9, 'A': 10, 'A+': 11
}

# Shot profile grade (SPG) and defensive shot profile grade (DSPG) metrics
grade_mapping = {
    'F': 1,
    'D': 2,
    'C': 3,
    'B': 4,
    'A': 5
}

# Convert PosSize in 'Player Role Fit' sheet
if 'PosSize' in player_role_fit.columns:
    player_role_fit['PosSize_numeric'] = player_role_fit['PosSize'].map(possize_mapping)
    
    # Show what grades were found
    unique_vals = player_role_fit['PosSize'].dropna().unique()
    print(f"✓ PosSize: Converted to numeric (D=1 to A+=11)")
    print(f"  Found grades: {sorted([v for v in unique_vals if pd.notna(v)])}")

# Convert SPG in 'Team Skill Fit' sheet
if 'SPG' in team_skill_fit.columns:
    team_skill_fit['SPG_numeric'] = team_skill_fit['SPG'].map(grade_mapping)
    
    # Show what grades were found
    unique_vals = team_skill_fit['SPG'].dropna().unique()
    print(f"✓ SPG: Converted to numeric (F=1 to A=5)")
    print(f"  Found grades: {sorted([v for v in unique_vals if pd.notna(v)])}")

# Convert DSPG in 'Team Skill Fit' sheet  
if 'DSPG' in team_skill_fit.columns:
    team_skill_fit['DSPG_numeric'] = team_skill_fit['DSPG'].map(grade_mapping)
    
    # Show what grades were found
    unique_vals = team_skill_fit['DSPG'].dropna().unique()
    print(f"✓ DSPG: Converted to numeric (F=1 to A=5)")
    print(f"  Found grades: {sorted([v for v in unique_vals if pd.notna(v)])}")

# Player role fit standardization
print("\n" + "="*70)
print("STEP 3: Standardizing Player Role Fit Metrics")
print("="*70)

role_standardize_cols = [
    'Usage Rate (USG%)',
    'Touches / Game', 
    'GR', 
    'r3PAr',
    'Pace',
    'VR',
    'MD',
    'Length',
    'PORT',
    'PosSize_numeric'
]

# Calculate mean and standard deviation, then apply z-score formula
scaler = StandardScaler()

# Count many columns were successfully standardized
standardized_count = 0

# Standardization loop
    # Loop through each column
    # Check if column exists in the DataFrame
    # Only standardize actual values, not missing data
    # Calculate z-score
    # Create new column for these new values
for col in role_standardize_cols:
    if col in player_role_fit.columns:
        non_null_mask = player_role_fit[col].notna()
        non_null_count = non_null_mask.sum() 
        
        if non_null_count > 1:
            standardized = scaler.fit_transform(
                player_role_fit.loc[non_null_mask, [col]]
            )
            
            player_role_fit[f'{col}_std'] = np.nan
            player_role_fit.loc[non_null_mask, f'{col}_std'] = standardized.flatten()
            print(f"✓ {col:30s} | {non_null_count:4d} values standardized")
            standardized_count += 1
        else:
            print(f"✗ {col:30s} | Skipped (only {non_null_count} non-null values)")
    else:
        print(f"✗ {col:30s} | Column not found")

print(f"\n→ Player Role Fit: {standardized_count}/{len(role_standardize_cols)} metrics standardized")

# Player skill fit standardization
print("\n" + "="*70)
print("STEP 4: Standardizing Player Skill Fit Metrics")
print("="*70)

skill_standardize_cols = [
    'Isolation PPP',
    'Transition PPP',
    'PnR Ball Handler PPP',
    'PnR Roll Man PPP',
    'Post Up PPP',
    'Spot Up PPP',
    'Handoff PPP',
    'Cut PPP',
    'Off Screen PPP',    
    'rTS%',
    'SQ',
    'Catch & Shoot (3P%)',
    'Pull Up Shooting (eFG%)',
    'BC',
    'oLOAD',
    'PR',
    'LT_06_%',
    'PLUSMINUS at Rim',
    'Contested Shots per Game',
    'Deflections per Game',
    'raDRDB',
    'raDTOV',
    'OLEBRON',
    'DLEBRON',
    'LEBRON',
    'NetRTG',
    'PIE',
    'On/Off +/-'
]

# Count many columns were successfully standardized
standardized_count = 0

# Standardization loop
    # Loop through each column
    # Check if column exists in the DataFrame
    # Only standardize actual values, not missing data
    # Calculate z-score
    # Create new column for these new values
for col in skill_standardize_cols:
    if col in player_skill_fit.columns:
        non_null_mask = player_skill_fit[col].notna()
        non_null_count = non_null_mask.sum()
        
        if non_null_count > 1:
            # Apply z-score standardization
            standardized = scaler.fit_transform(
                player_skill_fit.loc[non_null_mask, [col]]
            )
            
            # Create new column with '_std' suffix
            player_skill_fit[f'{col}_std'] = np.nan
            player_skill_fit.loc[non_null_mask, f'{col}_std'] = standardized.flatten()
            
            print(f"✓ {col:30s} | {non_null_count:4d} values standardized")
            standardized_count += 1
        else:
            print(f"✗ {col:30s} | Skipped (only {non_null_count} non-null values)")
    else:
        print(f"✗ {col:30s} | Column not found")

print(f"\n→ Player Skill Fit: {standardized_count}/{len(skill_standardize_cols)} metrics standardized")

# Team role fit standardization
print("\n" + "="*70)
print("STEP 5: Standardizing Team Role Fit Metrics")
print("="*70)

team_role_standardize_cols = [
    'Isolation', 
    'Transition', 
    'PnR Ball Handler', 
    'PnR Roll Man',
    'Post Up', 
    'Spot Up', 
    'Handoff', 
    'Cut', 
    'Off Screen',    
    'Pace',
    'Team Passer Rating',
    'AST%',
    'Restricted Area FGM', 
    'Restricted Area FGA', 
    'Restricted Area FG%',
    'In the Paint FGM', 
    'In the Paint FGA', 
    'In the Paint FG%',
    'Mid Range FGM', 
    'Mid Range FGA', 
    'Mid Range FG%',
    'Corner 3 FGM', 
    'Corner 3 FGA', 
    'Corner 3 FG%',
    'Above the Break 3 FGM', 
    'Above the Break 3 FGA', 
    'Above the Break 3 FG%',   
    'Opponent Restricted Area FGM', 
    'Opponent Restricted Area FGA',
    'Opponent Restricted Area FG%',
    'Opponent In the Paint FGM', 
    'Opponent In the Paint FGA',
    'Opponent In the Paint FG%',
    'Opponent Mid Range FGM', 
    'Opponent Mid Range FGA', 
    'Opponent Mid Range FG%',
    'Opponent Corner 3 FGM', 
    'Opponent Corner 3 FGA', 
    'Opponent Corner 3 FG%',
    'Opponent Above the Break 3 FGM', 
    'Opponent Above the Break 3 FGA',
    'Opponent Above the Break 3 FG%'
]

# Count many columns were successfully standardized
standardized_count = 0

# Standardization loop
    # Loop through each column
    # Check if column exists in the DataFrame
    # Only standardize actual values, not missing data
    # Calculate z-score
    # Create new column for these new values
for col in team_role_standardize_cols:
    if col in team_role_fit.columns:
        non_null_mask = team_role_fit[col].notna()
        non_null_count = non_null_mask.sum()
        
        if non_null_count > 1:
            standardized = scaler.fit_transform(
                team_role_fit.loc[non_null_mask, [col]]
            )
            
            team_role_fit[f'{col}_std'] = np.nan
            team_role_fit.loc[non_null_mask, f'{col}_std'] = standardized.flatten()
            
            print(f"✓ {col:40s} | {non_null_count:4d} values standardized")
            standardized_count += 1
        else:
            print(f"✗ {col:40s} | Skipped (only {non_null_count} non-null values)")
    else:
        print(f"✗ {col:40s} | Column not found")

print(f"\n→ Team Role Fit: {standardized_count}/{len(team_role_standardize_cols)} metrics standardized")

# Team skill fit standardization
print("\n" + "="*70)
print("STEP 6: Standardizing Team Skill Fit Metrics")
print("="*70)

team_skill_standardize_cols = [
    'Nrtg',
    'Isolation PPP', 
    'Transition PPP', 
    'PnR Ball Handler PPP',
    'PnR Roll Man PPP', 
    'Post Up PPP', 
    'Spot Up PPP',
    'Handoff PPP', 
    'Cut PPP', 
    'Off Screen PPP',
    'eFG%',
    '3P%',
    'SQ',
    'PTS_GAINED',
    'Ortg',
    'OREB%',
    'FTAr',
    'TOV %',
    'Drtg',
    'oEFG%',
    'PTS_SAVED',
    'DRB%',
    'oFTAr',
    'oTOV%',
    'oOREB%',
    'SPG_numeric',
    'DSPG_numeric'
]

# Count many columns were successfully standardized
standardized_count = 0

# Standardization loop
    # Loop through each column
    # Check if column exists in the DataFrame
    # Only standardize actual values, not missing data
    # Calculate z-score
    # Create new column for these new values
for col in team_skill_standardize_cols:
    if col in team_skill_fit.columns:
        non_null_mask = team_skill_fit[col].notna()
        non_null_count = non_null_mask.sum()
        
        if non_null_count > 1:
            standardized = scaler.fit_transform(
                team_skill_fit.loc[non_null_mask, [col]]
            )
            
            team_skill_fit[f'{col}_std'] = np.nan
            team_skill_fit.loc[non_null_mask, f'{col}_std'] = standardized.flatten()
            
            print(f"✓ {col:30s} | {non_null_count:4d} values standardized")
            standardized_count += 1
        else:
            print(f"✗ {col:30s} | Skipped (only {non_null_count} non-null values)")
    else:
        print(f"✗ {col:30s} | Column not found")

print(f"\n→ Team Skill Fit: {standardized_count}/{len(team_skill_standardize_cols)} metrics standardized")

# Save to new Excel file
print("\n" + "="*70)
print("STEP 7: Saving standardized data to NEW file")
print("="*70)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    player_role_fit.to_excel(writer, sheet_name='Player Role Fit', index=False)
    player_skill_fit.to_excel(writer, sheet_name='Player Skill Fit', index=False)
    team_role_fit.to_excel(writer, sheet_name='Team Role Fit', index=False)
    team_skill_fit.to_excel(writer, sheet_name='Team Skill Fit', index=False)

print(f"✓ Standardized data saved to:")
print(f"  {output_file}")
print(f"\n✓ Original file remains unchanged:")
print(f"  {input_file}")

# Summary
print("\n" + "="*70)
print("Summary")
print("="*70)

# Count columns with '_std' suffix (standardized z-scores)
role_std_count = len([c for c in player_role_fit.columns if '_std' in c])
skill_std_count = len([c for c in player_skill_fit.columns if '_std' in c])
team_role_std_count = len([c for c in team_role_fit.columns if '_std' in c])

# Count columns with '_norm' suffix (normalized 0-1 scores)
team_role_norm_count = len([c for c in team_role_fit.columns if '_norm' in c])

# Count standardized columns in team skill fit
team_skill_std_count = len([c for c in team_skill_fit.columns if '_std' in c])

# Display summary for each sheet
print(f"\nPlayer Role Fit:")
print(f"  • {role_std_count} metrics standardized (Z-score)")
print(f"  • Original + standardized columns preserved")

print(f"\nPlayer Skill Fit:")
print(f"  • {skill_std_count} metrics standardized (Z-score)")
print(f"  • Original + standardized columns preserved")

print(f"\nTeam Role Fit:")
print(f"  • {team_role_std_count} metrics standardized (Z-score)")
print(f"  • {team_role_norm_count} composite scores normalized (0-1)")
print(f"  • Original + processed columns preserved")

print(f"\nTeam Skill Fit:")
print(f"  • {team_skill_std_count} metrics standardized (Z-score)")
print(f"  • Original + standardized columns preserved")

# Calculate total metrics processed across all sheets
total_processed = role_std_count + skill_std_count + team_role_std_count + team_skill_std_count + team_role_norm_count
print(f"\n{'='*70}")
print(f"TOTAL: {total_processed} metrics processed")
print(f"{'='*70}")

# Example

print("\n" + "="*70)
print("Example")
print("="*70)

# Show example metrics
if 'Minutes per Game' in player_role_fit.columns and 'Usage Rate (USG%)_std' in player_role_fit.columns:
    # Show example of a raw (non-standardized) metric
    print("\nRaw Metric Example (Minutes per Game - LEFT RAW):")
    mpg_sample = player_role_fit['Minutes per Game'].dropna().head(5)
    print(f"  Sample values: {mpg_sample.tolist()}")
    print(f"  Mean: {player_role_fit['Minutes per Game'].mean():.2f}")
    print(f"  Std: {player_role_fit['Minutes per Game'].std():.2f}")
    
    # Show example of a standardized metric
    print("\nStandardized Metric Example (Usage Rate):")
    print(f"  Original Mean: {player_role_fit['Usage Rate (USG%)'].mean():.2f}")
    print(f"  Original Std: {player_role_fit['Usage Rate (USG%)'].std():.2f}")
    print(f"  Standardized Mean: {player_role_fit['Usage Rate (USG%)_std'].mean():.6f}")
    print(f"  Standardized Std: {player_role_fit['Usage Rate (USG%)_std'].std():.6f}")
    
    # Explain what these numbers mean
    print(f"\n  Interpretation:")
    print(f"    • Standardized mean ≈ 0 (centered at average)")
    print(f"    • Standardized std ≈ 1 (normalized spread)")
    print(f"    • Values show standard deviations from average")
    print(f"    • +1.0 = one standard deviation above average")
    print(f"    • -1.0 = one standard deviation below average")
    print(f"    • ±2.0 = significantly above/below average (elite/poor)")

# Complete
print("\n" + "="*70)
print("Standardization Complete!")
print("="*70)

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)